In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
from config import g_key
import sys
from datetime import datetime
import traceback
import gmaps

In [2]:
#Helper function to convert a yes input string to a True boolean value
def yesno_str_to_bool(user_str):
    return (user_str == "yes")

In [3]:
#Read the CSV into a dataframe
weather_df = pd.read_csv("analysis/WeatherPy_challenge.csv") 

In [4]:
#User Inputs
min_temp=float(input("What min temperature are you interested in?"))
max_temp=float(input("What max temperature are you interested in?"))
rain=input("Do you want it to be raining? (yes/no) ")
snow=input("Do you want it to be snowing? (yes/no) ")

What min temperature are you interested in?60
What max temperature are you interested in?90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [5]:
#Get initial boolean values
include_rain = yesno_str_to_bool(rain)
include_snow = yesno_str_to_bool(snow)

In [6]:
#Filter out the data between the min-max temperature ranges from the user input
valid_temp_range=weather_df[(weather_df["Max Temp"] >= min_temp) & (weather_df["Max Temp"] <= max_temp)]
valid_temp_range.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
2,samarai,pg,"04/19/2020, 23:06:14",-10.62,150.67,84.69,71,67,8.46,light rain,0.16,0.0
3,rikitea,pf,"04/19/2020, 23:06:15",-23.12,-134.97,76.80,71,61,6.62,broken clouds,0.00,0.0
4,faanui,pf,"04/19/2020, 23:02:14",-16.48,-151.75,82.96,76,89,21.74,light rain,0.24,0.0
5,mount gambier,au,"04/19/2020, 23:06:15",-37.83,140.77,60.80,63,75,14.99,broken clouds,0.00,0.0
8,hermanus,za,"04/19/2020, 23:06:16",-34.42,19.23,61.00,93,11,1.01,few clouds,0.00,0.0


In [7]:
#Filter based on rain in the last 3 hours
if include_rain:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]>0]
else:
    valid_data_rain = valid_temp_range[valid_temp_range["Rain inches (last 3 hrs)"]==0]

In [8]:
valid_data_rain.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
3,rikitea,pf,"04/19/2020, 23:06:15",-23.12,-134.97,76.80,71,61,6.62,broken clouds,0.0,0.0
5,mount gambier,au,"04/19/2020, 23:06:15",-37.83,140.77,60.80,63,75,14.99,broken clouds,0.0,0.0
8,hermanus,za,"04/19/2020, 23:06:16",-34.42,19.23,61.00,93,11,1.01,few clouds,0.0,0.0
13,airai,pw,"04/19/2020, 23:06:17",-8.93,125.41,72.64,47,30,3.29,scattered clouds,0.0,0.0
16,nelson bay,au,"04/19/2020, 23:06:18",-32.72,152.15,71.60,35,99,12.75,overcast clouds,0.0,0.0


In [9]:
#Additional Filter based on snow in the last 3 hours
if include_snow:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]>0]
else:
    valid_data = valid_data_rain[valid_data_rain["Snow inches (last 3 hrs)"]==0]

In [10]:
valid_data.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
3,rikitea,pf,"04/19/2020, 23:06:15",-23.12,-134.97,76.80,71,61,6.62,broken clouds,0.0,0.0
5,mount gambier,au,"04/19/2020, 23:06:15",-37.83,140.77,60.80,63,75,14.99,broken clouds,0.0,0.0
8,hermanus,za,"04/19/2020, 23:06:16",-34.42,19.23,61.00,93,11,1.01,few clouds,0.0,0.0
13,airai,pw,"04/19/2020, 23:06:17",-8.93,125.41,72.64,47,30,3.29,scattered clouds,0.0,0.0
16,nelson bay,au,"04/19/2020, 23:06:18",-32.72,152.15,71.60,35,99,12.75,overcast clouds,0.0,0.0


In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = valid_data[["City", "Country", "Max Temp", "Current Description","Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,rikitea,pf,76.80,broken clouds,-23.12,-134.97,
5,mount gambier,au,60.80,broken clouds,-37.83,140.77,
8,hermanus,za,61.00,few clouds,-34.42,19.23,
13,airai,pw,72.64,scattered clouds,-8.93,125.41,
16,nelson bay,au,71.60,overcast clouds,-32.72,152.15,
23,kavaratti,in,89.60,few clouds,10.57,72.64,
24,port alfred,za,72.00,clear sky,-33.59,26.89,
25,buta,cd,72.72,overcast clouds,2.79,24.73,
29,bambous virieux,mu,75.99,light intensity shower rain,-20.34,57.76,
31,kapaa,us,77.00,broken clouds,22.08,-159.32,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    lat_long = f"{lat},{lng}"
    params["location"] = lat_long
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    if len(hotels["results"]) == 0:
        print(f"Hotel results == 0... skipping. {lat_long}")
        continue
    try:
        hotel_result = hotels["results"]
        hotel_df.loc[index, "Hotel Name"] = hotel_result[0]["name"]
    except (IndexError):
        print(f"Hotel not found... skipping. {hotel_result}")
    except:
        print(f"Unexpected error: {sys.exc_info()[0]}")
        print(f"Unexpected error[1]: {sys.exc_info()[1]}")

Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. -37.83,140.77
Hotel results == 0... skipping. -34.42,19.23
Hotel results == 0... skipping. -8.93,125.41
Hotel results == 0... skipping. -32.72,152.15
Hotel results == 0... skipping. 10.57,72.64
Hotel results == 0... skipping. -33.59,26.89
Hotel results == 0... skipping. 2.79,24.73
Hotel results == 0... skipping. -20.34,57.76
Hotel results == 0... skipping. 22.08,-159.32
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. 3.07,172.79
Hotel results == 0... skipping. -21.2,-175.2
Hotel results == 0... skipping. -20.41,57.7
Hotel results == 0... skipping. -38.0,-57.56
Hotel results == 0... skipping. 29.16,16.14
Hotel results == 0... skipping. -13.88,167.55
Hotel results == 0... skipping. 37.47,57.33
Hotel results == 0... skipping. -33.59,26.89
Hotel results == 0... skipping. -13.7,-76.22
Hotel results == 0... skipping. -22.27,28.75
Hotel results == 0... skipping. 14.69,-17.44
Hotel res

Hotel results == 0... skipping. 22.29,114.16
Hotel results == 0... skipping. 22.08,-159.32
Hotel results == 0... skipping. -32.72,152.15
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. 39.49,16.96
Hotel results == 0... skipping. 22.29,114.16
Hotel results == 0... skipping. -33.93,18.42
Hotel results == 0... skipping. -20.41,57.7
Hotel results == 0... skipping. 22.89,-109.91
Hotel results == 0... skipping. -23.34,-57.06
Hotel results == 0... skipping. -38.0,-57.56
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. 19.73,-155.09
Hotel results == 0... skipping. 52.66,90.09
Hotel results == 0... skipping. -23.65,-70.4
Hotel results == 0... skipping. 24.19,102.82
Hotel results == 0... skipping. -11.7,43.26
Hotel results == 0... skipping. -1.2,-46.02
Hotel results == 0... skipping. -33.93,18.42
Hotel results == 0... skipping. 20.2,1.01
Hotel results == 0... skipping. -11.1,-77.61
Hotel r

Hotel results == 0... skipping. -0.72,8.78
Hotel results == 0... skipping. 19.21,-69.34
Hotel results == 0... skipping. 26.1,-80.4
Hotel results == 0... skipping. -33.59,26.89
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. 19.22,-104.7
Hotel results == 0... skipping. -33.59,26.89
Hotel results == 0... skipping. -0.74,-90.35
Hotel results == 0... skipping. -34.42,19.23
Hotel results == 0... skipping. -33.73,-70.75
Hotel results == 0... skipping. 13.09,-1.08
Hotel results == 0... skipping. 20.89,-156.47
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. 54.95,73.04
Hotel results == 0... skipping. -1.87,11.06
Hotel results == 0... skipping. -30.18,-50.21
Hotel results == 0... skipping. -34.42,19.23
Hotel results == 0... skipping. -23.12,-134.97
Hotel results == 0... skipping. -33.02,27.91
Hotel results == 0... skipping. 32.48,-96.99
Hotel results == 0... skipping. -29.95,-71.34
Hotel results == 0... skipping. -35.12,139.27
Hotel 

In [14]:
#Set HTML template for the popup on each marker on GMaps
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Descritpion</dt><dd>Description: {Current Description} with a max temp: {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]

In [16]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#Ensure there is at least 1 location and apply the info boxes to each pin.
if len(locations) > 0:
    # Get the maximum temperature.
    max_temp = hotel_df["Max Temp"]
    #GMaps weights doesn't allow negative values.
    temps = []
    for temp in max_temp:
        temps.append(max(temp, 0))

    #heat_layer = gmaps.heatmap_layer(locations,weights=temps,dissipating=False,max_intensity=300, point_radius=4)
    heat_layer = gmaps.heatmap_layer(locations,weights=temps)
    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)
    #Save hotel dataframe to a .CSV
    hotel_df.to_csv('./analysis/WeatherPy_vacation.csv')

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [17]:
hotel_df.head(15)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,rikitea,pf,76.80,broken clouds,-23.12,-134.97,
5,mount gambier,au,60.80,broken clouds,-37.83,140.77,
8,hermanus,za,61.00,few clouds,-34.42,19.23,
13,airai,pw,72.64,scattered clouds,-8.93,125.41,
16,nelson bay,au,71.60,overcast clouds,-32.72,152.15,
23,kavaratti,in,89.60,few clouds,10.57,72.64,
24,port alfred,za,72.00,clear sky,-33.59,26.89,
25,buta,cd,72.72,overcast clouds,2.79,24.73,
29,bambous virieux,mu,75.99,light intensity shower rain,-20.34,57.76,
31,kapaa,us,77.00,broken clouds,22.08,-159.32,
